# Modeleringsprosjekt 2020

# Oppagve 5: Numerisk integrasjon

Integrasjon er en metode i mattematikk mankan bruke på å finne arealet under grafen.  I analytisk matte bruker man anti-derivasjon til å finne dette arealet, men i numerisk matte er det flere utlike metoder.  I numerisk matte holder man seg kun til bestemte integraler.  Dette betyr at man integrerer en funksjon i et interval.  Da finner man arealet under grafen til funksjonen i dette intervallet.  Under skal vi se ulike metoder for å gjøre dette.

## Rektangelmetoden

### Teori

Rektangelmetoden gåut ut på at man setter rektangler under grafen med en liten bredde og en høyde tilsvarende funksjonsverdien i et punkt.  Legger man sammen arealet av disse rektanglene får man noe som tilsvarer arealet under grafen.  Og hvis man lar antall rektangler gå mot uendelig vil arealet av dem nærme seg arealet under grafen.  Denne metoden for å finne det bestemte integralet kales Riemann sum.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c9/LeftRiemann2.svg/1280px-LeftRiemann2.svg.png" alt="Riemann sum illustrasjon" style="width: 200px; float: left"/>
Figuren er en illustrasjon av at rektangler under grafen kan tilnærme arealet under grafen.

Det mattematiske uttrykket for dette vil være slik:

$$\int_{a}^{b} {f(x) dx} \approx \lim_{N \rightarrow \infty} (\Delta x \sum_{i = 0}^{N} {f(a + i * \Delta x)})$$

Der:

$$\Delta x = \frac{b - a}{N}$$

I disse uttrykkene representerer $N$ antall rektangler, $a$ er starten på det bestemte integralet, $b$ er slutten på det bestemte integralet og $\Delta x$ er bredden på rektangelene.

I summen over kunne vi mulitplisert med $\Delta x$ med hvert ledde i summen. Da ville vi lagt sammen alle arealene, men sånn det er skrevet nå legger vi sammen høyden i alle rektanglene og multipliserer med bredden etter på.  Dette gjør det enklere å regne og får koden til å gå bitte litt raskere.

### Kode

In [1]:
def RektangelMetoden(f, a, b, N = 100000):
    # f : En funksjon å integrere
    # a : Starten på det bestemte integralet
    # b : Slutten på det bestemte integralet
    # N : Antall rektangler
    
    # Bredden per rektangel
    dx = (b - a) / N
    
    # Arealet under grafen
    A = 0
    
    # Loop gjennom alle rekangelene
    for i in range(N):
        # Legg sammen høyden i rektanglene
        A += f(a + i * dx)
    
    # Multipliser med bredden per integral for å finne arealet
    A *= dx

    # returner arealet under grafen f
    return A

Dette kan vi teste med en enkel funksjon som $f(x) = x^2$

In [2]:
def f(x):
    return x**2

Areal = RektangelMetoden(f, 1, 2)
print(Areal)

2.333318333350027


Dette svaret er nærme det analytiske svaret som er på $\frac{7}{3} \approx 2.3333333333333$

### Ulikt antall rektangler
Hvor nærme vi kommer avgjøres av funksjonen og hvor mange rektangler vi har.  Under prøver vi andre funksjoner med ulikt antall rektangler.

In [3]:
def g(x):
    return 3 * x - 5

def h(x):
    return 0.1 * 2 ** x

a = 1
b = 3

print("Antall rektangler | f(x)       | g(x)       | h(x) ")
print("------------------+------------+------------+------------")

for i in range(7):
    rektangler = 10 ** i
    fverdi = RektangelMetoden(f, a, b, rektangler)
    gverdi = RektangelMetoden(g, a, b, rektangler)
    hverdi = RektangelMetoden(h, a, b, rektangler)
    
    print("{:>17} | {:<10.7f} | {:<10.7f} | {:<10.7f} ".format(rektangler, fverdi, gverdi, hverdi))

Antall rektangler | f(x)       | g(x)       | h(x) 
------------------+------------+------------+------------
                1 | 2.0000000  | -4.0000000 | 0.4000000  
               10 | 7.8800000  | 1.4000000  | 0.8070029  
              100 | 8.5868000  | 1.9400000  | 0.8596309  
             1000 | 8.6586680  | 1.9940000  | 0.8650172  
            10000 | 8.6658667  | 1.9994000  | 0.8655570  
           100000 | 8.6665867  | 1.9999400  | 0.8656110  
          1000000 | 8.6666587  | 1.9999940  | 0.8656164  


De analytiske svaret for f(x) er $\frac{26}{3} \approx 8.666666667$, for g(x) er det $2$ og for h(x) er det $\approx 0.8656170245333$

Vi ser tydelig at desto flere rektangler vi har desto nærmere komme vi det analytiske svaret.  Likevel på grunn av at float tall kan bli upresiste med mange desimaler, kan det hende at hvis $\Delta x$ blir alt for liten kan den bli mindr epresis igjen.

### Variasjoner av metoden

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/45/RightRiemann2.svg/1280px-RightRiemann2.svg.png" style="width: 200px; float: left;">For å forbedre metoden kan man flytte på punktet der man måler høyden.  Firguren til venstre er et eksempel der man måler høyden på grafen på høyre siden av rektangelet i steden for på venstre siden.  Ved å kombinere ulike metoder kan mna få et enda mer presist resultat.  En kode med støtte for dette kan se slik ut

In [5]:
def RektangelMetoden(f, a, b, N = 100000, regel = "venstre"):
    # f : En funksjon å integrere
    # a : Starten på det bestemte integralet
    # b : Slutten på det bestemte integralet
    # N : Antall rektangler
    # regel : Hvor man skal regne ut høyden ["venstre" (Standard) | "midten" | "hoyre" | float mellom 0 og 1]

    # Regne ut hva man må legge til i for å regne høyden på riktig sted
    i_regel = 0
    if type(regel) == float and 0 <= regel and regel <= 1:
        i_regel = regel
    elif regel.lower() == "midten":
        i_regel = 0.5
    elif regel.lower() == "hoyre":
        i_regel = 1
    elif regel.lower() != "venstre":
        # Kast en feil melding hvis regel argumentet er brukt feil
        raise Exception(f"Regel må være [\"venstre\" (Standard) | \"midten\" | \"hoyre\" | float mellom 0 og 1], men det ble spesifisert {regel}")

    # Bredden per rektangel
    dx = (b - a) / N
    # Arealet under grafen
    A = 0

    # Loop gjennom alle rekangelene
    for i in range(N):
        # Legg sammen høyden i rektanglene
        A += f(a + (i + i_regel) * dx)

    # Multipliser med bredden per integral for å finne arealet
    A *= dx

    # returner arealet under grafen f
    return A

Over kan vi se at om vi spesifiserer venstre får vi samme resultat som tidligere, men hvis vi spesifiserer midten måler vi på midten, eller på høyre side hvis vi spesiferer hoyre.  Under kan vi se resultatet av dette.

In [6]:
print("Venstre:", RektangelMetoden(f, 0, 1, regel = "venstre"))
print("Midten :", RektangelMetoden(f, 0, 1, regel = "midten"))
print("Hoyre  :", RektangelMetoden(f, 0, 1, regel = "hoyre"))

Venstre: 0.3333283333499996
Midten : 0.3333333333249928
Hoyre  : 0.3333383333499996


Her ser vi at resultatet er veldig likt, men dersom vi senker antall rektangler får vi ganske ulike svar.

In [7]:
print("Venstre:", RektangelMetoden(f, 0, 1, 5, "venstre"))
print("Midten :", RektangelMetoden(f, 0, 1, 5, "midten"))
print("Hoyre  :", RektangelMetoden(f, 0, 1, 5, "hoyre"))

Venstre: 0.24000000000000005
Midten : 0.33000000000000007
Hoyre  : 0.44000000000000006
